In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('seaborn')

from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/tesla-stock-price/Tesla.csv - Tesla.csv.csv')
df.shape

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date'].head()

In [ ]:
plt.figure(figsize=(10,10))
sns.pairplot(data=df, kind='scatter', corner=True)
plt.show()

# **Feature Scaling**

In [ ]:
df.set_index('Date',inplace=True)
df.head()

In [ ]:
scale = MinMaxScaler()
scalled = scale.fit_transform(df)

In [ ]:
i = 0
for col in df.columns:
    df[col] = scalled[:,i]
    i += 1

In [ ]:
df.head()

In [ ]:
df.describe()

# **Data Splitting and training**

In [ ]:
x, y = df.drop('Adj Close', axis=1), df['Adj Close']

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
y_train.shape, y_test.shape

# **Model Building and Predictions**

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(5, input_shape=(5,), activation='linear'),
    keras.layers.Dense(1, activation='linear')
])

model.compile(
    loss='mean_squared_error',
    optimizer='adam'
)

In [ ]:
model.fit(x_train, y_train, epochs=50)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_pred_test = model.predict(x_test).flatten()
y_pred_train = model.predict(x_train).flatten()

# **Actual vs predicted**

In [ ]:
def actual_vs_predicted(y_true, y_pred, title=None, xscale='linear',yscale='linear', return_correlation=False):
    x= 'Actual'
    y= 'Predicted'
    plt.figure(figsize=(10,10))
    ap = pd.DataFrame({x:y_true,y:y_pred})
    sns.scatterplot(data=ap,x=x, y=y)
    plt.title(title)
    plt.xscale(xscale) #Linear, log and symlog
    plt.yscale(yscale) #Linear, log and symlog
    plt.show()
    if return_correlation:
        return ap.corr()

In [ ]:
actual_vs_predicted(y_test, y_pred_test, title="Y test Data.",return_correlation=True)

In [ ]:
r2_score(y_pred_test, y_test)

In [ ]:
actual_vs_predicted(y_train, y_pred_train, title="Y Train Data.")

In [ ]:
r2_score(y_pred_train, y_train)

# **Compare Actual vs predicted**

In [ ]:
def plot_actual_vs_predicted(y_true, y_pred, title=None, xscale='linear',yscale='linear'):
    x= 'Actual'
    y= 'Predicted'
    plt.figure(figsize=(10,10))
    ap = pd.DataFrame({x:y_true,y:y_pred})
    ap.sort_values(by=x,ascending=True,inplace=True)
    plt.scatter(np.arange(len(ap[x])),ap[x],color='blue', label=x, linewidth=0.9, marker='*')
    plt.plot(np.arange(len(ap[x])),ap[y],color='red', label=y, linewidth=2)
    
    plt.title(title)
    plt.legend()
    plt.xscale(xscale) #Linear, log and symlog
    plt.yscale(yscale) #Linear, log and symlog
    plt.show()


In [ ]:
plot_actual_vs_predicted(y_test,y_pred_test,"Comparing Test Actual vs predicted")

In [ ]:
plot_actual_vs_predicted(y_train,y_pred_train,"Comparing Train Actual vs predicted")

# **Saving Model as file**

In [ ]:
model.save('Tesla-Stock-Model')

In [ ]:
!ls